In [0]:
# Auto Loader - Triggered execution for batch processing
# Configuration using dbutils widgets for flexibility

from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

# Create dbutils widgets for configuration
dbutils.widgets.text("catalog", "samples", "Catalog Name")
dbutils.widgets.text("schema", "default", "Schema Name")
dbutils.widgets.text("volume", "autoloader_volume", "Volume Name")

# Get configuration values
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
volume = dbutils.widgets.get("volume")

spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")

# Volume paths
volume_path = f"/Volumes/{catalog}/{schema}/{volume}"
checkpoint_base_path = f"/Volumes/{catalog}/{schema}/checkpoints"
schema_base_path = f"/Volumes/{catalog}/{schema}/{volume}/schemas"

print(f"Configuration:")
print(f"Catalog: {catalog}")
print(f"Schema: {schema}")
print(f"Volume: {volume}")
print(f"Volume Path: {volume_path}")

add_table = "bronze_add"
contact_table = "contact_silver"
contact_segment_table = "contact_segment_table"
remove_table = "bronze_remove"


In [0]:
add_files_schema = StructType([
    StructField("Name", StringType(), True),
    StructField("email", StringType(), False),
    StructField("DOB", DateType(), True),
    StructField("occupation", StringType(), True),
    StructField("phone_number", StringType(), True)
])


# Create add data table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {add_table} (
  Name STRING,
  email STRING,
  DOB DATE,
  occupation STRING,
  phone_number STRING,
  source_file STRING,
  ingestion_timestamp TIMESTAMP,
  file_modification_time TIMESTAMP
)
""")

In [0]:
add_source_path = f"{volume_path}/*/add/*.csv"
add_checkpoint_path = f"{checkpoint_base_path}/add_stream"
batch_id = datetime.now().strftime("%Y%m%d_%H%M%S")

print(f"Processing ADD files from: {add_source_path}")
print(f"Batch ID: {batch_id}")

add_df = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .schema(add_files_schema)
    .option("cloudFiles.maxFilesPerTrigger", "1000")
    
    # CSV-specific options
    .option("header", "true")
    .option("inferSchema", "true")
    
    .load(add_source_path)
    .withColumn("source_file", col("_metadata.file_path"))
    .withColumn("ingestion_timestamp", current_timestamp())
    .withColumn("file_modification_time", col("_metadata.file_modification_time"))
)

# Process with triggered execution (availableNow)
add_query = (add_df
    .writeStream
    .outputMode("append")
    .option("checkpointLocation", add_checkpoint_path)
    .option("mergeSchema", "true")
    .trigger(availableNow=True) 
    .toTable(add_table)
)

# Wait for completion
add_query.awaitTermination()
print("✓ ADD files processed successfully")


In [0]:
# create the scd silver table
from pyspark.sql import Window
from delta.tables import DeltaTable

contact_checkpoint_path = f"{checkpoint_base_path}/contact"
# Create historical contact data
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {contact_table} (
  Name STRING,
  email STRING,
  DOB DATE,
  occupation STRING,
  phone_number STRING
)
""")

target_columns = ["Name", "DOB", "occupation", "phone_number"] 

def upsert_to_contacts(microbatch_df, _):
    # pick latest per id inside the microbatch
    w = Window.partitionBy("email").orderBy(col("file_modification_time").desc())
    latest = (microbatch_df
              .withColumn("rn", row_number().over(w))
              .filter("rn = 1")
              .drop("rn"))

    tgt = DeltaTable.forName(spark, "contact_silver")
    
    # TODO - check that the update is newer than the existing value 
    # Update condition: only update if source has non-null values that are different from target
    update_conditions = []
    for c in target_columns:
        update_conditions.append(f"(s.{c} IS NOT NULL AND (t.{c} IS NULL OR t.{c} != s.{c}))")
    update_cond = " OR ".join(update_conditions) if update_conditions else "false"
    
    # Update set: only update columns where source is not null
    update_set = {}
    for c in target_columns:
        update_set[c] = when(col(f"s.{c}").isNotNull(), col(f"s.{c}")).otherwise(col(f"t.{c}"))
    
    # Insert values: use source values, including email as the key
    insert_vals = {c: col(f"s.{c}") for c in ["email"] + target_columns}
    
    (tgt.alias("t")
        .merge(latest.alias("s"), "t.email = s.email")
        .whenMatchedUpdate(condition=update_cond, set=update_set)
        .whenNotMatchedInsert(values=insert_vals)
        .execute())

bronze_df = (spark.readStream
  .table(add_table)
)

writer = (bronze_df.writeStream
  .foreachBatch(upsert_to_contacts)
  .option("checkpointLocation", contact_checkpoint_path)
  .trigger(availableNow=True)
)

q = writer.start()
q.awaitTermination()

In [0]:
from pyspark.sql.functions import split, element_at, col, row_number
from pyspark.sql import Window
from delta.tables import DeltaTable

contact_segment_checkpoint = f"{checkpoint_base_path}/contact_segment"
# Create historical contact data
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {contact_segment_table} (
  email STRING,
  segment STRING,
  file_modification_time TIMESTAMP,
  is_active BOOLEAN
) USING DELTA
""")

bronze_df = (spark.readStream
  .table(add_table)
)

bronze_df = bronze_df.select(
        "email",
        "file_modification_time",
        element_at(split("source_file", "/"), -3).alias("segment")
    )

target_columns = ["email", "segment"] 

def upsert_to_contact_segment(microbatch_df, _):
    w = Window.partitionBy("email", "segment").orderBy(col("file_modification_time").desc())
    latest = (microbatch_df
              .withColumn("rn", row_number().over(w))
              .filter("rn = 1")
              .drop("rn"))

    tgt = DeltaTable.forName(spark, contact_segment_table)
    
    # Insert values: insert s.email and s.segment
    insert_vals = {'email': col('s.email'), 'segment': col('s.segment'), "file_modification_time": col('s.file_modification_time'), "is_active": lit(True)}

    (tgt.alias("t")
        .merge(latest.alias("s"), "t.email = s.email AND t.segment = s.segment AND t.file_modification_time < s.file_modification_time")
        .whenNotMatchedInsert(values=insert_vals)
        .execute())

writer = (bronze_df.writeStream
  .foreachBatch(upsert_to_contact_segment)
  .option("checkpointLocation", contact_segment_checkpoint)
  .trigger(availableNow=True)
)

q = writer.start()
q.awaitTermination()

In [0]:
# Create remove data table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {remove_table} (
  email STRING,
  source_file STRING,
  ingestion_timestamp TIMESTAMP,
  file_modification_time TIMESTAMP
)
""")

remove_source_path = f"{volume_path}/*/remove/*.csv"
remove_checkpoint_path = f"{checkpoint_base_path}/remove_stream"
remove_schema_path = schema_base_path+"/remove_schema"
batch_id = datetime.now().strftime("%Y%m%d_%H%M%S")

print(f"Processing REMOVE files from: {remove_source_path}")
print(f"Batch ID: {batch_id}")


remove_df = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", remove_schema_path)
    
    # CSV-specific options
    .option("header", "true")
    .option("inferSchema", "true")

    .load(remove_source_path)
    .select("email")
    .withColumn("source_file", col("_metadata.file_path"))
    .withColumn("ingestion_timestamp", current_timestamp())
    .withColumn("file_modification_time", col("_metadata.file_modification_time"))
)

# Process with triggered execution (availableNow)
remove_query = (remove_df
    .writeStream
    .outputMode("append")
    .option("checkpointLocation", remove_checkpoint_path)
    .option("mergeSchema", "true")
    .trigger(availableNow=True) 
    .toTable(remove_table)
)

# Wait for completion
remove_query.awaitTermination()
print("✓ REMOVE files processed successfully")

In [0]:
from pyspark.sql.functions import split, element_at, col, row_number
from pyspark.sql import Window
from delta.tables import DeltaTable

contact_segment_table = "contact_segment_table"
contact_segment_rm_checkpoint = f"{checkpoint_base_path}/contact_segment_remove"

bronze_remove_df = (spark.readStream
  .table(remove_table)
)

bronze_remove_df = bronze_remove_df.select(
        "email",
        "file_modification_time",
        element_at(split("source_file", "/"), -3).alias("segment")
    )

target_columns = ["email", "segment"] 

def remove_from_contact_segment(microbatch_df, _):
    w = Window.partitionBy("email", "segment").orderBy(col("file_modification_time").desc())
    latest = (microbatch_df
              .withColumn("rn", row_number().over(w))
              .filter("rn = 1")
              .drop("rn"))

    tgt = DeltaTable.forName(spark, contact_segment_table)
    
    # Insert soft delete values: delete s.email and s.segment
    delete_vals = {'email': col('s.email'), 'segment': col('s.segment'), "file_modification_time": col('s.file_modification_time'), "is_active": lit(False)}
    update_cond = "t.file_modification_time < s.file_modification_time"

    (tgt.alias("t")
        .merge(latest.alias("s"), "t.email = s.email AND t.segment = s.segment")
        .whenMatchedUpdate(condition=update_cond, set=delete_vals)
        .execute())

writer = (bronze_remove_df.writeStream
  .foreachBatch(remove_from_contact_segment)
  .option("checkpointLocation", contact_segment_rm_checkpoint)
  .trigger(availableNow=True)
)

q = writer.start()
q.awaitTermination()

In [0]:
# Utility functions for managing Auto Loader

def trigger_autoloader_processing():
    """
    Function to trigger Auto Loader processing for both ADD and REMOVE files.
    This can be called manually or scheduled as needed.
    """
    batch_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    print(f"🚀 Starting Auto Loader processing - Batch ID: {batch_id}")
    
    try:
        # This would contain the processing logic from cells 3-5
        print("\n1️⃣ Processing ADD files...")
        print("\n2️⃣ Processing REMOVE files...")
        print("\n3️⃣ Updating incremental tables...")
        
        print(f"\n✅ Batch {batch_id} completed successfully!")
        return batch_id
        
    except Exception as e:
        print(f"❌ Error in batch {batch_id}: {str(e)}")
        return None

def reset_checkpoints():
    """
    Reset Auto Loader checkpoints to reprocess all files.
    Use with caution - this will cause all files to be reprocessed.
    """
    try:
        dbutils.fs.rm(f"{checkpoint_base_path}", True)
        print("✅ Checkpoints reset successfully")
        print("⚠️  Next run will reprocess ALL files")
    except Exception as e:
        print(f"❌ Error resetting checkpoints: {str(e)}")

def view_checkpoint_status():
    """
    View the current checkpoint status and processed files.
    """
    try:
        # Check if checkpoint directories exist
        try:
            add_checkpoint_files = dbutils.fs.ls(f"{checkpoint_base_path}/add_stream")
            add_checkpoint_count = len(add_checkpoint_files)
        except:
            add_checkpoint_count = 0
            
        try:
            remove_checkpoint_files = dbutils.fs.ls(f"{checkpoint_base_path}/remove_stream")
            remove_checkpoint_count = len(remove_checkpoint_files)
        except:
            remove_checkpoint_count = 0
        
        print(f"📂 ADD Checkpoint Status: {add_checkpoint_count} files")
        print(f"📂 REMOVE Checkpoint Status: {remove_checkpoint_count} files")
        
        # Show latest processed files
        try:
            latest_processed = spark.sql(f"""
                SELECT file_type, COUNT(*) as file_count, MAX(processed_timestamp) as last_processed
                FROM {processed_files_table}
                GROUP BY file_type
            """)
            
            print("\n📈 Processing Summary:")
            latest_processed.display()
        except:
            print("\n📈 No processing history available yet")
        
    except Exception as e:
        print(f"❌ Error checking status: {str(e)}")

# Example usage functions
print("🛠️  Utility functions available:")
print("- trigger_autoloader_processing(): Run processing manually")
print("- view_checkpoint_status(): Check processing status")
print("- reset_checkpoints(): Reset to reprocess all files (use with caution)")
print("\n📝 Usage Examples:")
print("# batch_id = trigger_autoloader_processing()")
print("# view_checkpoint_status()")
print("# reset_checkpoints()  # Use with caution!")
print("\n📅 Scheduling Options:")
print("- Run this notebook on a schedule using Databricks Jobs")
print("- Call trigger_autoloader_processing() from another notebook")
print("- Use Databricks Workflows to orchestrate processing")

In [0]:
view_checkpoint_status()

In [0]:
display(spark.sql(f"DROP TABLE IF EXISTS {add_table}"))
display(spark.sql(f"DROP TABLE IF EXISTS {contact_table}"))
display(spark.sql(f"DROP TABLE IF EXISTS {contact_segment_table}"))
display(spark.sql(f"DROP TABLE IF EXISTS {remove_table}"))
display(spark.sql(f"DROP TABLE IF EXISTS main_table"))
display(spark.sql(f"DROP TABLE IF EXISTS segments")) 
dbutils.fs.rm(checkpoint_base_path, True)
